In [21]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

import random
from user_agent import generate_user_agent, generate_navigator

In [14]:
actors = ["Bruce Willis", "Tom Cruise", "Uma Thurman", "Bill Cosby", "Bruce Willis", "Tom Cruise", "Uma Thurman", "Bill Cosby","Bruce Willis", "Tom Cruise", "Uma Thurman", "Bill Cosby", "Bruce Willis", "Tom Cruise", "Uma Thurman", "Bill Cosby"]
#do mod 20 and print out to keep us updated how far it is through

In [35]:
chrome_path = "./chromedriver"

options = webdriver.ChromeOptions()
options.add_argument('--load-extension=./adblock4Sellenium/3.34.0_0')
options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})


options.add_argument("--user-agent=" + 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')


driver = webdriver.Chrome(chrome_path, chrome_options=options)

# generate_user_agent(os=('mac', 'linux'))


/Users/school/Desktop/Fall 2018 UW Classes/Data Sci - Info 370 HW/Labs/theEnv/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
#creating a dataframe
actorDataFrame = []

In [ ]:
#Finds number of movies from soup
def findNumMovies(soup):
        numCreds = soup.find("div", {"id": "filmo-head-actor"})
        if numCreds is None:
             numCreds = soup.find("div", {"id": "filmo-head-actress"})
        rawNumOfMovies = numCreds.text
        return rawNumOfMovies

In [ ]:
    
    #under: call function to web scrape the page
    def scrape(actorName, htmlURL):
        #try:
            html_doc = requests.get(htmlURL)
            soup = BeautifulSoup(html_doc.text, 'html.parser')

            awards = soup.findAll("span", {"class": "awards-blurb"})
            rawWinsAndNoms = awards[1].text
            # winsAndNoms = re.findall('\s\d+\s', rawWinsAndNoms)
            winsAndNoms = re.findall('\d+', rawWinsAndNoms)

            numWins = int(winsAndNoms[0])
            numNoms = int(winsAndNoms[1])
            rawNumOfMovies = findNumMovies(soup)
            
            
            rawNumOfMovies = re.findall('\d+', rawNumOfMovies)
            numMovies = int(rawNumOfMovies[0])

            # List of the Rating Values (in order; number of wins, num of nominations, and num of movies):
            actorValues = [numWins, numNoms, numMovies]
            actorDataFrame.append({'actorName': actorName, 'actorMovieCount': numMovies,
                                                                 'actorAwardNominations': numNoms, 'actorAwardWins': numWins})
            
        #except:
           # print('Error. Skipping to next actor.')



In [ ]:
count = len(actors)

driver.get('https://www.imdb.com/?ref_=nv_home')
for name in actors:
    if count % 2 == 0:
        print("number of actors remaining: %i" % count)
    #going to main page of IMDB
    #Find search bar, clear and search for actor
    searchBar = driver.find_element_by_id('navbar-query')
    searchBar.clear()
    searchBar.send_keys(name)
    searchBar.send_keys(Keys.RETURN)
    assert "No results found." not in driver.page_source
    try:
        #Taking the top actor from the list of names that show up from the search
        personWanted = driver.find_element_by_class_name('findResult')
        personURL = personWanted.find_element_by_tag_name('a')
        personURL.click()
        driver.current_url
        scrape(name, driver.current_url)
    #except instance when a search of an actor has a non-actor (ex: show) as the top choice
    except:
        #Reason for doing a whole repeat is because of HTML format where cannot determine if link to click on is actor or not
        #Find search bar, clear and search for actor
        searchBar = driver.find_element_by_id('navbar-query')
        searchBar.clear()
        searchBar.send_keys(name)
        searchBar.send_keys(Keys.RETURN)
        assert "No results found." not in driver.page_source
        #Taking the top actor from the list of names that show up from the search
        moreName = driver.find_element_by_partial_link_text('More name matches')
        moreName.click()
        personWanted = driver.find_element_by_class_name('findResult')
        personURL = personWanted.find_element_by_tag_name('a')
        personURL.click()
        driver.current_url
        scrape(name, driver.current_url)
    count -= 1
    
actorDataFrame = pd.DataFrame(actorDataFrame, columns = ['actorName', 'actorMovieCount', 'actorAwardNominations', 'actorAwardWins'] )
actorDataFrame.to_csv('actors.csv', index = False)
actorDataFrame
print("its done")